
# Bayesian Optimization with Retrieval Optimizer

This notebook demonstrates how to use Bayesian optimization to tune Redis-based retrieval pipelines. Unlike a grid study—which tests all combinations—Bayesian optimization intelligently searches the configuration space, prioritizing promising settings based on previous results. This is especially useful when the number of possible configurations is large and exhaustive search would be too costly.

You'll define a study configuration, specify embedding models and search methods, and let the optimizer guide the search toward the best-performing retrieval setup.


# Installation

In [ ]:
%pip install redis-retrieval-optimizer

## Dataset

We'll import a dataset from the [beir benchmark IR project](https://github.com/beir-cellar/beir) to get going quickly. 

In [3]:
# Load data
from redis_retrieval_optimizer.corpus_processors import eval_beir

# check the link above for different datasets to try
beir_dataset_name = "nfcorpus"
data_folder = "data"

# Load sample data
corpus, queries, qrels = eval_beir.get_beir_dataset(beir_dataset_name)

/Users/robert.shelton/.pyenv/versions/3.12.8/lib/python3.12/site-packages/beir/util.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


16:01:33 beir.datasets.data_loader INFO   Loading Corpus...


  0%|          | 0/3633 [00:00<?, ?it/s]

16:01:34 beir.datasets.data_loader INFO   Loaded 3633 TEST Documents.
16:01:34 beir.datasets.data_loader INFO   Doc Example: {'text': 'Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast cancer recurrence but the effect on disease-specific mortality remains unclear. We evaluated risk of breast cancer death among statin users in a population-based cohort of breast cancer patients. The study cohort included all newly diagnosed breast cancer patients in Finland during 1995–2003 (31,236 cases), identified from the Finnish Cancer Registry. Information on statin use before and after the diagnosis was obtained from a national prescription database. We used the Cox proportional hazards regression method to estimate mortality among statin users with statin use as time-dependent variable. A total of 4,151 participants had used statins. During the median follow-up of 3.25 years after the diagnosis (rang

Now that we have our data we will save it locally to the gitignored `data/` folder

In [6]:
import os

os.makedirs(data_folder, exist_ok=True)

In [7]:
import json

with open(f"data/{beir_dataset_name}_corpus.json", "w") as f:
    json.dump(corpus, f)

with open(f"data/{beir_dataset_name}_queries.json", "w") as f:
    json.dump(queries, f)

with open(f"data/{beir_dataset_name}_qrels.json", "w") as f:
    json.dump(qrels, f)

## Study config

In this directory there is a yaml file containing a configuration for a bayesian study that looks like this:

```yaml
# path to data files for easy read
corpus: "data/nfcorpus_corpus.json"
queries: "data/nfcorpus_queries.json"
qrels: "data/nfcorpus_qrels.json"

index_settings:
  name: "optimize"
  vector_field_name: "vector" # name of the vector field to search on
  text_field_name: "text" # name of the text field for lexical search
  from_existing: false
  vector_dim: 384 # should match first embedding model or from_existing
  additional_fields:
      - name: "title"
        type: "text"

optimization_settings:
  # defines the options optimization can take
  metric_weights:
    f1_at_k: 1
    embedding_latency: 1
    total_indexing_time: 1
  algorithms: ["hnsw"]
  vector_data_types: ["float16", "float32"]
  distance_metrics: ["cosine"]
  n_trials: 10
  n_jobs: 1
  ret_k: [1, 10] # potential range of value to be sampled during study
  ef_runtime: [10, 20, 30, 50]
  ef_construction: [100, 150, 200, 250, 300]
  m: [8, 16, 64]


search_methods: ["vector", "lin_combo"]
embedding_models:
  - type: "hf"
    model: "sentence-transformers/all-MiniLM-L6-v2"
    dim: 384
    embedding_cache_name: "vec-cache" # avoid names with including 'ret-opt' as this can cause collisions
    dtype: "float32"
```

## Running a study

To run a study simple pass the path to config, redis_url, and corpus processing function to the `run_bayes_study` function and the package will take care of the rest. 

In [1]:
import os
from redis_retrieval_optimizer.bayes_study import run_bayes_study
from redis_retrieval_optimizer.corpus_processors import eval_beir
from dotenv import load_dotenv

# load environment variables containing necessary credentials
load_dotenv()

redis_url = os.environ.get("REDIS_URL", "redis://localhost:6379/0")

metrics = run_bayes_study(
    config_path="bayes_study_config.yaml",
    redis_url=redis_url,
    corpus_processor=eval_beir.process_corpus
)

[I 2025-07-03 16:03:34,383] A new study created in memory with name: test


16:03:34 redisvl.index.index INFO   Index already exists, not overwriting.
16:03:36 datasets INFO   PyTorch version 2.7.1 available.
16:03:36 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
16:03:36 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

16:03:38 root INFO   Data indexed total_indexing_time=3.803s, num_docs=3633
16:03:42 root INFO   Saving metrics for study: a796dacc-52e2-4fc7-be2c-3a7246d10901, METRICS={'search_method': ['hybrid'], 'total_indexing_time': [3.803], 'avg_query_time': [0.0035999603684841665], 'model': ['sentence-transformers/all-MiniLM-L6-v2'], 'model_dim': [384], 'ret_k': [5], 'recall': [0.15498797328057845], 'ndcg': [0.20277849135854045], 'f1': [0.13071176354799932], 'precision': [0.24334365325077406], 'algorithm': ['hnsw'], 'ef_construction': [250], 'ef_runtime': [30], 'm': [8], 'distance_metric': ['cosine'], 'vector_data_type': ['float16'], 'objective_value': [-0.9455885555010966]}


[I 2025-07-03 16:03:42,133] Trial 0 finished with value: -0.9455885555010966 and parameters: {'model_info': {'type': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384, 'embedding_cache_name': 'vec-cache', 'dtype': 'float32'}, 'search_method': 'hybrid', 'algorithm': 'hnsw', 'var_dtype': 'float16', 'distance_metric': 'cosine', 'ret_k': 5, 'ef_runtime': 30, 'ef_construction': 250, 'm': 8}. Best is trial 0 with value: -0.9455885555010966.


...

16:04:35 redisvl.index.index INFO   Index already exists, not overwriting.
16:04:35 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
16:04:35 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

16:04:36 root INFO   Data indexed total_indexing_time=3.103s, num_docs=3633
16:04:38 root INFO   Saving metrics for study: a796dacc-52e2-4fc7-be2c-3a7246d10901, METRICS={'search_method': ['hybrid', 'vector', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'vector', 'vector', 'vector', 'hybrid', 'vector', 'vector', 'vector', 'hybrid', 'hybrid', 'vector', 'vector', 'hybrid', 'vector', 'hybrid'], 'total_indexing_time': [3.803, 3.803, 3.803, 3.803, 2.745, 2.745, 4.533, 2.328, 2.328, 2.328, 3.103, 3.103, 3.103, 3.103, 3.103, 3.103, 3.103, 3.103, 3.103, 3.103], 'avg_query_time': [0.0035999603684841665, 0.001152925077975719, 0.0036754940316403983, 0.0036588128506214626, 0.0023498874699737263, 0.002616440917685305, 0.0013452875355817954, 0.0007695502171944539, 0.0009129800294574938, 0.0027583870725365984, 0.0013001275136374838, 0.0013324815791457799, 0.0015705347799295243, 0.0040682447214982825, 0.0036695534968892857, 0.0015897462980666028, 0.0013265897615037097, 0.0036797398015072473, 0.0013631526899

[I 2025-07-03 16:04:38,204] Trial 19 finished with value: -1.016615599267783 and parameters: {'model_info': {'type': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384, 'embedding_cache_name': 'vec-cache', 'dtype': 'float32'}, 'search_method': 'hybrid', 'algorithm': 'hnsw', 'var_dtype': 'float16', 'distance_metric': 'cosine', 'ret_k': 6, 'ef_runtime': 50, 'ef_construction': 250, 'm': 16}. Best is trial 6 with value: -0.9211616263746649.


16:04:38 root INFO   Completed Bayesian optimization... 


16:04:38 root INFO   Best Configuration: 6: {'model_info': {'type': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384, 'embedding_cache_name': 'vec-cache', 'dtype': 'float32'}, 'search_method': 'vector', 'algorithm': 'hnsw', 'var_dtype': 'float16', 'distance_metric': 'cosine', 'ret_k': 4, 'ef_runtime': 30, 'ef_construction': 200, 'm': 64}:


16:04:38 root INFO   Best Score: [-0.9211616263746649]




In [2]:
metrics[["search_method", "algorithm", "vector_data_type", "ef_construction", "ef_runtime", "m", "avg_query_time", "total_indexing_time", "recall", "precision", "ndcg", "f1", "objective_value"]].sort_values(by="objective_value", ascending=False)

,search_method,algorithm,vector_data_type,ef_construction,ef_runtime,m,avg_query_time,total_indexing_time,recall,precision,ndcg,f1,objective_value
6,vector,hnsw,float16,200,30,64,0.001345,4.533,0.152715,0.243034,0.193432,0.129629,-0.921162
2,hybrid,hnsw,float16,300,20,8,0.003675,3.803,0.154988,0.243344,0.202778,0.130712,-0.945439
3,hybrid,hnsw,float16,150,10,64,0.003659,3.803,0.154988,0.243344,0.202778,0.130712,-0.945472
0,hybrid,hnsw,float16,250,30,8,0.003600,3.803,0.154988,0.243344,0.202778,0.130712,-0.945589
13,hybrid,hnsw,float16,150,30,8,0.004068,3.103,0.154988,0.243344,0.202778,0.130712,-1.015701
17,hybrid,hnsw,float16,300,30,64,0.003680,3.103,0.154988,0.243344,0.202778,0.130712,-1.016472
14,hybrid,hnsw,float16,100,50,16,0.003670,3.103,0.154988,0.243344,0.202778,0.130712,-1.016492
19,hybrid,hnsw,float16,250,50,16,0.003607,3.103,0.154988,0.243344,0.202778,0.130712,-1.016616
1,vector,hnsw,float16,200,20,64,0.001153,3.803,0.143791,0.238080,0.181270,0.124497,-1.038827
5,hybrid,hnsw,float32,200,30,64,0.002616,2.745,0.154988,0.243344,0.202778,0.130712,-1.065182
